In [1]:
%pip install mlflow==1.30
%pip install boto3

     |████████████████████████████████| 17.0 MB 1.5 MB/s eta 0:00:01
  Attempting uninstall: mlflow
    Found existing installation: mlflow 1.28.0
    Uninstalling mlflow-1.28.0:
      Successfully uninstalled mlflow-1.28.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install feast[cassandra]==0.26

Note: you may need to restart the kernel to use updated packages.


In [4]:
import mlflow

In [5]:
mlflow.__version__

'1.30.0'

In [ ]:
from feast import FeatureStore
store = FeatureStore()

In [13]:
from datetime import datetime, timedelta
store.materialize(
    feature_views=["credit_history"],
    start_date=datetime.utcnow() - timedelta(days=1000), end_date=datetime.utcnow() - timedelta(minutes=10)
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Materializing 1 feature views from 2020-10-07 07:07:45+00:00 to 2023-07-04 06:57:45+00:00 into the cassandra online store.

credit_history:


100%|█████████████████████████████████████████████████████████| 28633/28633 [06:23<00:00, 74.74it/s]


In [11]:
from datetime import datetime, timedelta
store.materialize(
    feature_views=["zipcode_features"],
    start_date=datetime.utcnow() - timedelta(days=3650), end_date=datetime.utcnow() - timedelta(minutes=10)
)

Materializing 1 feature views from 2013-07-06 07:02:30+00:00 to 2023-07-04 06:52:30+00:00 into the cassandra online store.

zipcode_features:


100%|████████████████████████████████████████████████████████| 28844/28844 [04:19<00:00, 111.31it/s]


In [12]:
store.materialize(
    feature_views=["client_features"],
    start_date=datetime.utcnow() - timedelta(days=3650), end_date=datetime.utcnow() - timedelta(minutes=10)
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Materializing 1 feature views from 2013-07-06 07:06:50+00:00 to 2023-07-04 06:56:50+00:00 into the cassandra online store.

client_features:


100%|████████████████████████████████████████████████████████| 28633/28633 [00:48<00:00, 593.32it/s]


In [7]:

from sklearn import tree
from sklearn import ensemble
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from mlflow.models.signature import infer_signature
from mlflow.models import MetricThreshold
import pandas as pd
import dill
import sklearn
import json

from feast import FeatureStore

store = FeatureStore(repo_path=".")
dataset_name = 'loan_approval_baseline'
training_df = store.get_saved_dataset(dataset_name).to_df()

# cfg = {"type": "feast_dataset", "fs_path": ".", "dataset_path": "loan_approval_baseline", "categorical_features": ["person_home_ownership", "loan_intent", "state"], "cols_to_drop": ["created_timestamp", "event_timestamp", "loan_id", "zipcode", "dob_ssn", "city", "location_type"]}

# import json
# with open("cfg.json", "w") as f:
#     json.dump(cfg, f)

encoder = OrdinalEncoder()
categorical_features = [
    "person_home_ownership",
    "loan_intent",
    "state",
]
cols_to_drop = ['created_timestamp', 'event_timestamp', 'loan_id', 'zipcode', 'dob_ssn', 'city', 'location_type']

encoder.fit(training_df[categorical_features])
transform_training_df=training_df.copy()
transform_training_df[categorical_features] = encoder.transform(
    training_df[categorical_features]
)

target = "loan_status"
train_X = transform_training_df[
    transform_training_df.columns.drop(target)
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("person_income")
    .drop("zipcode")
    .drop("dob_ssn")
    .drop("city")
    .drop("location_type")
]
train_X = train_X.reindex(sorted(train_X.columns), axis=1)
train_Y = transform_training_df.loc[:, target]

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.10)

n_estimators = 100
classifier = ensemble.GradientBoostingClassifier(n_estimators=n_estimators)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_true=y_test, y_pred = predictions)

def eval_metrics(actual, pred):
    precision = precision_score(actual, pred)
    recall = recall_score(actual, pred)
    f1 = f1_score(actual, pred)
    return precision, recall, f1

(precision, recall, f1) = eval_metrics(y_test, predictions)
print(precision)
print(recall)
print(f1)

import joblib

model_name = "model.bin"
encoder_name = "encoder.bin"

joblib.dump(classifier, model_name)
joblib.dump(encoder, encoder_name)

artifacts = {
    "model": model_name,
    "encoder": encoder_name,
}

data_cfg_name = "data_cfg.json"

data_cfg = {
    'type': 'feast_dataset',
    'fs_path': ".",
    'dataset_path': dataset_name,
    'categorical_features': categorical_features,
    'cols_to_drop': cols_to_drop,
}

with open(data_cfg_name, 'w') as fp:
    json.dump(data_cfg, fp)

target_column = 'loan_status'
eval_data = x_test
eval_data[target_column] = y_test

eval_train_df = training_df.copy()
eval_train_df=eval_train_df[
    eval_train_df.columns
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("person_income")
    .drop("city")
    .drop("location_type")
]

signature = infer_signature(x_test, predictions)

wrapped_signature = infer_signature

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import logging

class ModelWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import joblib
        self.encoder = joblib.load(context.artifacts["encoder"])            
        self.model = joblib.load(context.artifacts["model"]) 
        self.categorical_features = [
            "person_home_ownership",
            "loan_intent",
            "state",
        ]

    def predict(self, context, inp):

        import pandas as pd

        logger = logging.getLogger(__name__)

        if isinstance(inp, dict):
            features_df = pd.DataFrame.from_dict(inp)
        elif isinstance(inp, pd.DataFrame):
            features_df = inp

        logger.debug("Enriching features DataFrame: %s", features_df)

        # Apply ordinal encoding to categorical features
        features_df[self.categorical_features] = self.encoder.transform(
            features_df[self.categorical_features]
        )

        logger.debug("features DataFrame after encoging: %s", features_df)

        # Sort columns
        features_df = features_df.reindex(sorted(features_df.columns), axis=1)

        # Drop unnecessary columns
        df_for_predict = features_df.copy()
        if "city" in df_for_predict.columns:
            df_for_predict = df_for_predict[df_for_predict.columns.drop("city").drop("location_type")]

        if "zipcode" in df_for_predict.columns:
            df_for_predict = df_for_predict[df_for_predict.columns.drop("zipcode").drop("dob_ssn")]

        logger.debug("features DataFrame before predict: %s", df_for_predict)

        # Make prediction
#         df_for_predict["prediction"] = self.model.predict(df_for_predict)
        
        result = self.model.predict(df_for_predict)

#         result = pd.merge(features_df, df_for_predict, left_index=True, right_index=True)[["zipcode", "dob_ssn", "prediction"]]

#         logger.debug("result of predict: %s", result)

        # return result of credit scoring
#         return result.to_numpy()
        return result
#                 return df_for_predict.to_numpy()

conda_env={
    'channels': ['defaults'],
    'dependencies': [
      'python=3.8.10',
      'pip',
      {
        'pip': [
          'mlflow=={}'.format(mlflow.__version__),
          'numpy==1.23.4',
          'scikit-learn=={}'.format(sklearn.__version__),
          'joblib=={}'.format(joblib.__version__),
          'dill=={}'.format(dill.__version__),
#           'numpy==1.19',
        ],
      },
    ],
    'name': 'demo_env'
}

thresholds = {
#     "mean_ks_test": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "min_ks_test": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "mean_chi_square": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "min_chi_square": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
    "accuracy_score": MetricThreshold(
        threshold=0.05,
        higher_is_better=True,
    ),
}

mlflow.set_experiment("loans_demo_dev2")

with mlflow.start_run(run_name='train baseline model') as current_run:

    mlflow.log_param("n_estimators", n_estimators)

    mlflow.set_tag('base_dataset', dataset_name)

    model_without_wrapper = mlflow.sklearn.log_model(
        classifier, "model",  
        conda_env=conda_env, 
        signature=signature
    )
    
#     mlflow.log_artifact(data_cfg_name)

    result = mlflow.evaluate(
        model_without_wrapper.model_uri,
        eval_data,
        targets=target_column,
        model_type="classifier",
        evaluators=["default"],
        validation_thresholds=thresholds,
    )

    with mlflow.start_run(run_name='train baseline model', nested=True) as current_run:
        model_info = mlflow.pyfunc.log_model(
            artifact_path="model", 
            python_model=ModelWrapper(),
            artifacts=artifacts,
            conda_env=conda_env,
#                     signature=signature
        )
        
        mlflow.log_artifact(data_cfg_name)

        result = mlflow.evaluate(
            model_info.model_uri,
            eval_train_df,
            targets=target_column,
            model_type="classifier",
            evaluators=["default"],
            validation_thresholds=thresholds,
        )

    run_id = current_run.info.run_id
    print(run_id)


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1235: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


0.8218181818181818
0.576530612244898
0.6776611694152924


<ipython-input-7-8dbdac5da7da>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data[target_column] = y_test
/opt/conda/lib/python3.8/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

f168c543ca034fee82de8e22f28166d8


<Figure size 1050x700 with 0 Axes>

In [8]:
mlflow.pyfunc.get_model_dependencies(model_uri)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'model_uri' is not defined

In [2]:
%pip install feast[cassandra]==0.26

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     |████████████████████████████████| 3.7 MB 19.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install psycopg2-binary

     |████████████████████████████████| 3.0 MB 26.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [4]:
pd.read_parquet("loan_approval_2022q3.parquet").dtypes

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


loan_id                    int64
dob_ssn                   object
zipcode                    int64
person_age                 int64
person_income              int64
person_home_ownership     object
person_emp_length        float64
loan_intent               object
loan_amnt                  int64
loan_int_rate            float64
loan_status                int64
event_timestamp           object
created_timestamp         object
city                      object
state                     object
location_type             object
tax_returns_filed          int64
population                 int64
total_wages                int64
credit_card_due            int64
mortgage_due               int64
student_loan_due           int64
vehicle_loan_due           int64
hard_pulls                 int64
missed_payments_2y         int64
missed_payments_1y         int64
missed_payments_6m         int64
bankruptcies               int64
dtype: object

In [3]:
loans = pd.read_parquet("loan_table.parquet")

In [8]:
loans.columns

Index(['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status', 'event_timestamp',
       'created_timestamp'],
      dtype='object')

In [4]:
q3 = pd.read_parquet("loan_approval_baseline.parquet")[loans.columns]

In [12]:
loan_features = store.get_feature_service('loan_features')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [53]:
merged

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,17075,19650824_3778,36685,24,49000,OTHER,1.0,PERSONAL,20000,11.34,1,2021-05-27 16:21:00.214000+00:00,2021-05-27 16:21:00.214000+00:00
1,12347,19660819_6708,49024,24,33996,MORTGAGE,8.0,VENTURE,4000,14.91,0,2021-07-26 22:38:08.282000+00:00,2021-07-26 22:38:08.282000+00:00
2,14152,19510119_9702,98001,26,32000,RENT,3.0,DEBTCONSOLIDATION,5000,7.49,0,2021-07-03 22:29:23.311000+00:00,2021-07-03 22:29:23.311000+00:00
3,37394,19720314_5774,47601,39,55600,RENT,0.0,HOMEIMPROVEMENT,11500,13.22,0,2020-09-10 16:48:33.571000+00:00,2020-09-10 16:48:33.571000+00:00
4,29120,19691006_4715,42160,34,52000,RENT,1.0,EDUCATION,6000,12.53,1,2020-12-25 03:48:32.691000+00:00,2020-12-25 03:48:32.691000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,17739,19770117_1251,26241,24,60000,MORTGAGE,0.0,DEBTCONSOLIDATION,6000,5.42,0,2021-05-19 05:14:01.720000+00:00,2021-05-19 05:14:01.720000+00:00
9565,22354,19951008_5271,62051,26,75000,RENT,4.0,EDUCATION,14000,11.11,1,2021-03-21 09:30:52.943000+00:00,2021-03-21 09:30:52.943000+00:00
9566,27955,19460702_7893,31721,29,40000,MORTGAGE,13.0,PERSONAL,6400,7.51,1,2021-01-09 00:10:48.753000+00:00,2021-01-09 00:10:48.753000+00:00
9567,29804,19890819_1324,28398,30,41000,RENT,0.0,PERSONAL,7200,7.49,0,2020-12-16 10:34:29.544000+00:00,2020-12-16 10:34:29.544000+00:00


In [52]:
training_df = training_features.to_df()

training_df

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,...,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies,incomeamount12m
0,17075,19650824_3778,36685,24,49000,OTHER,1.0,PERSONAL,20000,11.34,...,621,978600,7411,28473,9,5,2,0,2,184860.0
1,12347,19660819_6708,49024,24,33996,MORTGAGE,8.0,VENTURE,4000,14.91,...,1361,1906718,31737,11861,3,4,2,0,0,216648.0
2,14152,19510119_9702,98001,26,32000,RENT,3.0,DEBTCONSOLIDATION,5000,7.49,...,7239,1364807,42571,19716,1,3,0,1,0,236940.0
3,37394,19720314_5774,47601,39,55600,RENT,0.0,HOMEIMPROVEMENT,11500,13.22,...,6339,809494,18703,2743,6,3,1,1,1,137124.0
4,29120,19691006_4715,42160,34,52000,RENT,1.0,EDUCATION,6000,12.53,...,1012,521320,18904,7387,7,7,3,0,0,187068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,17739,19770117_1251,26241,24,60000,MORTGAGE,0.0,DEBTCONSOLIDATION,6000,5.42,...,7234,1902427,10252,14150,4,6,2,1,0,141144.0
9574,22354,19951008_5271,62051,26,75000,RENT,4.0,EDUCATION,14000,11.11,...,7457,1279380,40336,23407,2,0,2,0,0,146460.0
9575,27955,19460702_7893,31721,29,40000,MORTGAGE,13.0,PERSONAL,6400,7.51,...,4653,1690266,548,2489,0,1,0,0,0,135660.0
9576,29804,19890819_1324,28398,30,41000,RENT,0.0,PERSONAL,7200,7.49,...,6023,240747,9313,4271,1,0,1,1,0,200232.0


In [5]:
q3

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,24648,19490816_6592,44287,23,120000,MORTGAGE,5.0,VENTURE,6000,9.99,0,2021-02-20 03:47:05.196000+00:00,2021-02-20 03:47:05.196000+00:00
1,28499,19831120_6431,33620,27,45000,MORTGAGE,11.0,PERSONAL,3450,11.99,0,2021-01-02 01:46:18.180000+00:00,2021-01-02 01:46:18.180000+00:00
2,36222,19911026_9586,92110,36,39520,OWN,14.0,VENTURE,5000,5.79,0,2020-09-25 15:19:18.262000+00:00,2020-09-25 15:19:18.262000+00:00
3,15836,19630507_1213,13865,24,57000,RENT,0.0,VENTURE,6000,7.90,0,2021-06-12 11:21:27.493000+00:00,2021-06-12 11:21:27.493000+00:00
4,14019,19671129_3523,97523,23,124000,RENT,2.0,PERSONAL,10000,14.27,0,2021-07-05 15:10:27.256000+00:00,2021-07-05 15:10:27.256000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,23050,19720313_3006,71749,23,116004,MORTGAGE,1.0,PERSONAL,14000,12.99,0,2021-03-12 12:36:35.004000+00:00,2021-03-12 12:36:35.004000+00:00
9565,20561,19570213_9898,33417,24,78500,MORTGAGE,8.0,VENTURE,6000,7.49,0,2021-04-13 05:59:23.122000+00:00,2021-04-13 05:59:23.122000+00:00
9566,37356,19600118_8531,60108,37,77000,MORTGAGE,2.0,VENTURE,15000,12.29,0,2020-09-11 04:26:00.413000+00:00,2020-09-11 04:26:00.413000+00:00
9567,25458,19981011_8953,20601,23,41000,RENT,7.0,EDUCATION,4700,7.90,0,2021-02-09 20:00:26.732000+00:00,2021-02-09 20:00:26.732000+00:00


In [24]:
q3["loan_id"].values

array([16138, 21829, 21424, ..., 25226, 17912, 18163])

In [29]:
q3.sort_values('loan_id')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
6450,10002,19860413_2537,97039,25,9600,MORTGAGE,1.0,MEDICAL,5500,12.87,1,2021-08-25 19:57:58.896000+00:00,2021-08-25 19:57:58.896000+00:00
6103,10006,19650101_1416,73724,26,77100,RENT,8.0,EDUCATION,35000,12.42,1,2021-08-25 18:44:33.965000+00:00,2021-08-25 18:44:33.965000+00:00
280,10008,19880707_1475,37074,24,83000,RENT,8.0,PERSONAL,35000,8.90,1,2021-08-25 18:07:51.500000+00:00,2021-08-25 18:07:51.500000+00:00
6272,10009,19780808_9123,71260,21,10000,OWN,6.0,VENTURE,1600,14.74,1,2021-08-25 17:49:30.267000+00:00,2021-08-25 17:49:30.267000+00:00
4811,10010,19800608_4782,75109,22,85000,RENT,6.0,VENTURE,35000,10.37,1,2021-08-25 17:31:09.034000+00:00,2021-08-25 17:31:09.034000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,38357,19471002_8974,2191,38,35000,RENT,0.0,DEBTCONSOLIDATION,5600,11.11,0,2020-08-29 10:13:46.509000+00:00,2020-08-29 10:13:46.509000+00:00
4879,38358,19731019_4439,43840,38,59475,MORTGAGE,10.0,MEDICAL,18125,14.84,1,2020-08-29 09:55:25.276000+00:00,2020-08-29 09:55:25.276000+00:00
6029,38381,19581106_4477,68461,37,90000,MORTGAGE,11.0,DEBTCONSOLIDATION,4000,6.62,0,2020-08-29 02:53:16.925000+00:00,2020-08-29 02:53:16.925000+00:00
4518,38383,19681212_7610,32457,38,200000,MORTGAGE,0.0,DEBTCONSOLIDATION,3000,7.68,0,2020-08-29 02:16:34.459000+00:00,2020-08-29 02:16:34.459000+00:00


In [30]:
test = loans[loans["loan_id"].isin(q3["loan_id"].values)]

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [6]:
loan_times = loans[["loan_id", "event_timestamp", "created_timestamp"]]

In [35]:
loan_times

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,event_timestamp,created_timestamp
0,10000,2021-08-25 20:34:41.361000+00:00,2021-08-25 20:34:41.361000+00:00
1,10001,2021-08-25 20:16:20.128000+00:00,2021-08-25 20:16:20.128000+00:00
2,10002,2021-08-25 19:57:58.896000+00:00,2021-08-25 19:57:58.896000+00:00
3,10003,2021-08-25 19:39:37.663000+00:00,2021-08-25 19:39:37.663000+00:00
4,10004,2021-08-25 19:21:16.430000+00:00,2021-08-25 19:21:16.430000+00:00
...,...,...,...
28633,38633,2020-08-25 21:48:06.292000+00:00,2020-08-25 21:48:06.292000+00:00
28634,38634,2020-08-25 21:29:45.059000+00:00,2020-08-25 21:29:45.059000+00:00
28635,38635,2020-08-25 21:11:23.826000+00:00,2020-08-25 21:11:23.826000+00:00
28636,38636,2020-08-25 20:53:02.594000+00:00,2020-08-25 20:53:02.594000+00:00


In [7]:
q3_without_time = q3[['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status']]

In [40]:
pd.concat([q3_without_time.set_index('loan_id'),loan_times.set_index('loan_id')], axis=1, join='inner').reset_index()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,16138,19710923_6795,64730,23,19250,RENT,1.0,HOMEIMPROVEMENT,6250,16.82,1,2021-06-08 14:58:35.226000+00:00,2021-06-08 14:58:35.226000+00:00
1,21829,19540514_5083,23833,22,13200,RENT,0.0,VENTURE,2500,9.91,1,2021-03-28 02:06:40.096000+00:00,2021-03-28 02:06:40.096000+00:00
2,21424,19500909_5696,59935,23,85000,RENT,0.0,PERSONAL,12000,10.37,0,2021-04-02 05:59:59.327000+00:00,2021-04-02 05:59:59.327000+00:00
3,13908,19940306_9908,21117,21,55000,RENT,3.0,EDUCATION,4800,11.12,0,2021-07-07 01:07:44.083000+00:00,2021-07-07 01:07:44.083000+00:00
4,24649,19740620_7017,56345,22,49000,MORTGAGE,6.0,VENTURE,2400,5.79,0,2021-02-20 03:28:43.963000+00:00,2021-02-20 03:28:43.963000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8197,27514,19650213_4791,1469,32,31000,MORTGAGE,16.0,DEBTCONSOLIDATION,3000,7.51,1,2021-01-14 15:04:52.361000+00:00,2021-01-14 15:04:52.361000+00:00
8198,10870,19620320_5684,83209,24,56000,RENT,2.0,EDUCATION,18000,13.48,1,2021-08-14 18:26:48.937000+00:00,2021-08-14 18:26:48.937000+00:00
8199,25226,20010514_4088,68760,23,48000,OWN,8.0,VENTURE,6000,12.87,0,2021-02-12 18:58:32.712000+00:00,2021-02-12 18:58:32.712000+00:00
8200,17912,19570622_8275,44656,24,60000,MORTGAGE,5.0,DEBTCONSOLIDATION,14000,7.51,0,2021-05-17 00:18:48.468000+00:00,2021-05-17 00:18:48.468000+00:00


In [8]:
merged = q3_without_time.merge(loan_times, on="loan_id", how="left")

In [13]:
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source import (
    PostgreSQLSource, SavedDatasetPostgreSQLStorage
)

training_features = store.get_historical_features(
    entity_df=merged, 
    features=loan_features
)

dataset = store.create_saved_dataset(
    from_=training_features,
    name=f'loan_approval_baseline',
    storage=SavedDatasetPostgreSQLStorage(table_ref=f'loan_approval_baseline')
)

/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [84]:
merged

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,24648,19490816_6592,44287,23,120000,MORTGAGE,5.0,VENTURE,6000,9.99,0,2021-02-20 03:47:05.196000+00:00,2021-02-20 03:47:05.196000+00:00
1,28499,19831120_6431,33620,27,45000,MORTGAGE,11.0,PERSONAL,3450,11.99,0,2021-01-02 01:46:18.180000+00:00,2021-01-02 01:46:18.180000+00:00
2,36222,19911026_9586,92110,36,39520,OWN,14.0,VENTURE,5000,5.79,0,2020-09-25 15:19:18.262000+00:00,2020-09-25 15:19:18.262000+00:00
3,15836,19630507_1213,13865,24,57000,RENT,0.0,VENTURE,6000,7.90,0,2021-06-12 11:21:27.493000+00:00,2021-06-12 11:21:27.493000+00:00
4,14019,19671129_3523,97523,23,124000,RENT,2.0,PERSONAL,10000,14.27,0,2021-07-05 15:10:27.256000+00:00,2021-07-05 15:10:27.256000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,23050,19720313_3006,71749,23,116004,MORTGAGE,1.0,PERSONAL,14000,12.99,0,2021-03-12 12:36:35.004000+00:00,2021-03-12 12:36:35.004000+00:00
9565,20561,19570213_9898,33417,24,78500,MORTGAGE,8.0,VENTURE,6000,7.49,0,2021-04-13 05:59:23.122000+00:00,2021-04-13 05:59:23.122000+00:00
9566,37356,19600118_8531,60108,37,77000,MORTGAGE,2.0,VENTURE,15000,12.29,0,2020-09-11 04:26:00.413000+00:00,2020-09-11 04:26:00.413000+00:00
9567,25458,19981011_8953,20601,23,41000,RENT,7.0,EDUCATION,4700,7.90,0,2021-02-09 20:00:26.732000+00:00,2021-02-09 20:00:26.732000+00:00


In [5]:
import mlflow
logged_model = 'runs:/6d9b64c6032a45d4a84365f56d2dab4d/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.

# loaded_model.predict(training_df.drop(["person_income", "loan_status"]))

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


In [57]:
training_df.columns

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Index(['person_age', 'person_income', 'loan_amnt', 'loan_int_rate',
       'loan_intent', 'loan_status', 'zipcode', 'person_home_ownership',
       'person_emp_length', 'created_timestamp', 'dob_ssn', 'loan_id', 'city',
       'state', 'location_type', 'tax_returns_filed', 'population',
       'total_wages', 'credit_card_due', 'mortgage_due', 'student_loan_due',
       'vehicle_loan_due', 'hard_pulls', 'missed_payments_2y',
       'missed_payments_1y', 'missed_payments_6m', 'bankruptcies',
       'incomeamount12m', 'event_timestamp'],
      dtype='object')

In [60]:
training_df.drop(["person_income", "loan_status"], axis=1).dtypes

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


person_age                        int64
loan_amnt                         int64
loan_int_rate                   float64
loan_intent                      object
zipcode                           int64
person_home_ownership            object
person_emp_length               float64
created_timestamp        datetime64[ns]
dob_ssn                          object
loan_id                           int64
city                             object
state                            object
location_type                    object
tax_returns_filed                 int64
population                        int64
total_wages                       int64
credit_card_due                   int64
mortgage_due                      int64
student_loan_due                  int64
vehicle_loan_due                  int64
hard_pulls                        int64
missed_payments_2y                int64
missed_payments_1y                int64
missed_payments_6m                int64
bankruptcies                      int64


In [10]:
loaded_model.predict(training_df2[["zipcode","dob_ssn","tax_returns_filed","total_wages","state","population","student_loan_due","credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies","missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m","person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"]])

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array([1, 1, 0, ..., 0, 0, 0])

In [9]:
dataset_name = 'loan_approval_2022q4'
training_df2 = store.get_saved_dataset(dataset_name).to_df()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.


In [7]:
training_df2[["zipcode","dob_ssn","tax_returns_filed","location_type","city","total_wages","state","population","student_loan_due","credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies","missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m","person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"]]

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,zipcode,dob_ssn,tax_returns_filed,location_type,city,total_wages,state,population,student_loan_due,credit_card_due,...,hard_pulls,mortgage_due,missed_payments_2y,incomeamount12m,person_age,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate
0,64730,19710923_6795,3134,PRIMARY,BUTLER,85617594,MO,5629,4764,2308,...,5,1317248,4,204756.0,23,RENT,1.0,HOMEIMPROVEMENT,6250,16.82
1,23833,19540514_5083,991,PRIMARY,CHURCH ROAD,39171958,VA,1846,18115,5366,...,2,1014189,0,236280.0,22,RENT,0.0,VENTURE,2500,9.91
2,59935,19500909_5696,1475,PRIMARY,TROY,32636522,MT,2615,22988,9459,...,10,1212162,2,234888.0,23,RENT,0.0,PERSONAL,12000,10.37
3,21117,19940306_9908,25767,PRIMARY,OWINGS MILLS,1520024734,MD,43195,31004,8379,...,10,470349,0,230664.0,21,RENT,3.0,EDUCATION,4800,11.12
4,56345,19740620_7017,6979,PRIMARY,LITTLE FALLS,214528388,MN,12016,21548,2210,...,0,1892410,0,239304.0,22,MORTGAGE,6.0,VENTURE,2400,5.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1469,19650213_4791,3457,PRIMARY,TOWNSEND,175478972,MA,6266,7451,3109,...,6,624769,0,165612.0,32,MORTGAGE,16.0,DEBTCONSOLIDATION,3000,7.51
8210,83209,19620320_5684,296,PRIMARY,POCATELLO,4971841,ID,484,32363,658,...,4,1024135,7,146484.0,24,RENT,2.0,EDUCATION,18000,13.48
8211,68760,20010514_4088,498,PRIMARY,NIOBRARA,9981758,NE,895,26776,8410,...,3,471210,2,239724.0,23,OWN,8.0,VENTURE,6000,12.87
8212,44656,19570622_8275,1318,PRIMARY,MINERAL CITY,40440220,OH,2337,4479,338,...,6,243918,7,162600.0,24,MORTGAGE,5.0,DEBTCONSOLIDATION,14000,7.51


In [67]:
loans.columns

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Index(['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status', 'event_timestamp',
       'created_timestamp'],
      dtype='object')

In [ ]:
"person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"

In [72]:
training_df

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,person_age,person_income,loan_amnt,loan_int_rate,loan_intent,loan_status,zipcode,person_home_ownership,person_emp_length,created_timestamp,...,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies,incomeamount12m,event_timestamp
0,24,49000,20000,11.34,PERSONAL,1,36685,OTHER,1.0,2021-05-27 16:21:00.214,...,978600,7411,28473,9,5,2,0,2,184860.0,2021-05-27 16:21:00.214
1,24,33996,4000,14.91,VENTURE,0,49024,MORTGAGE,8.0,2021-07-26 22:38:08.282,...,1906718,31737,11861,3,4,2,0,0,216648.0,2021-07-26 22:38:08.282
2,26,32000,5000,7.49,DEBTCONSOLIDATION,0,98001,RENT,3.0,2021-07-03 22:29:23.311,...,1364807,42571,19716,1,3,0,1,0,236940.0,2021-07-03 22:29:23.311
3,39,55600,11500,13.22,HOMEIMPROVEMENT,0,47601,RENT,0.0,2020-09-10 16:48:33.571,...,809494,18703,2743,6,3,1,1,1,137124.0,2020-09-10 16:48:33.571
4,34,52000,6000,12.53,EDUCATION,1,42160,RENT,1.0,2020-12-25 03:48:32.691,...,521320,18904,7387,7,7,3,0,0,187068.0,2020-12-25 03:48:32.691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,24,60000,6000,5.42,DEBTCONSOLIDATION,0,26241,MORTGAGE,0.0,2021-05-19 05:14:01.720,...,1902427,10252,14150,4,6,2,1,0,141144.0,2021-05-19 05:14:01.720
9574,26,75000,14000,11.11,EDUCATION,1,62051,RENT,4.0,2021-03-21 09:30:52.943,...,1279380,40336,23407,2,0,2,0,0,146460.0,2021-03-21 09:30:52.943
9575,29,40000,6400,7.51,PERSONAL,1,31721,MORTGAGE,13.0,2021-01-09 00:10:48.753,...,1690266,548,2489,0,1,0,0,0,135660.0,2021-01-09 00:10:48.753
9576,30,41000,7200,7.49,PERSONAL,0,28398,RENT,0.0,2020-12-16 10:34:29.544,...,240747,9313,4271,1,0,1,1,0,200232.0,2020-12-16 10:34:29.544


In [ ]:
training_df3 = 

In [22]:
categorical_features = [
    "person_home_ownership",
    "loan_intent",
    "city",
    "state",
    "location_type",
]
numeric_features = [
    "zipcode","dob_ssn","tax_returns_filed","total_wages","population","student_loan_due",
    "credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies",
    "missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m",
    "person_age","person_emp_length","loan_amnt","loan_int_rate"
]
target = "loan_status"

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [23]:
categorical_features + numeric_features + [target]

['person_home_ownership',
 'loan_intent',
 'city',
 'state',
 'location_type',
 'zipcode',
 'dob_ssn',
 'tax_returns_filed',
 'total_wages',
 'population',
 'student_loan_due',
 'credit_card_due',
 'missed_payments_1y',
 'vehicle_loan_due',
 'bankruptcies',
 'missed_payments_6m',
 'hard_pulls',
 'mortgage_due',
 'missed_payments_2y',
 'incomeamount12m',
 'person_age',
 'person_emp_length',
 'loan_amnt',
 'loan_int_rate',
 'loan_status']

In [24]:
training_df[categorical_features + numeric_features + [target]]

,person_home_ownership,loan_intent,city,state,location_type,zipcode,dob_ssn,tax_returns_filed,total_wages,population,...,missed_payments_6m,hard_pulls,mortgage_due,missed_payments_2y,incomeamount12m,person_age,person_emp_length,loan_amnt,loan_int_rate,loan_status
0,MORTGAGE,VENTURE,WEST SALEM,OH,PRIMARY,44287,19490816_6592,3446,105044479,6131,...,0,0,402046,0,133428.0,23,5.0,6000,9.99,0
1,MORTGAGE,PERSONAL,TAMPA,FL,PRIMARY,33620,19831120_6431,326,6446048,392,...,0,0,1457001,1,197664.0,27,11.0,3450,11.99,0
2,OWN,VENTURE,SAN DIEGO,CA,PRIMARY,92110,19911026_9586,12108,489992429,18233,...,0,1,1116961,1,134019.0,36,14.0,5000,5.79,0
3,RENT,VENTURE,WINDSOR,NY,PRIMARY,13865,19630507_1213,2857,92436837,5164,...,0,2,1986074,4,136392.0,24,0.0,6000,7.90,0
4,RENT,PERSONAL,CAVE JUNCTION,OR,PRIMARY,97523,19671129_3523,2481,38582611,4309,...,1,2,1166941,0,154164.0,23,2.0,10000,14.27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9570,MORTGAGE,PERSONAL,JUNCTION CITY,AR,PRIMARY,71749,19720313_3006,1152,36329410,2213,...,0,9,960101,7,228108.0,23,1.0,14000,12.99,0
9571,MORTGAGE,VENTURE,WEST PALM BEACH,FL,PRIMARY,33417,19570213_9898,12750,293168875,19843,...,0,1,1612387,0,135288.0,24,8.0,6000,7.49,0
9572,MORTGAGE,VENTURE,BLOOMINGDALE,IL,PRIMARY,60108,19600118_8531,11725,603291761,19903,...,0,2,478039,6,168237.0,37,2.0,15000,12.29,0
9573,RENT,EDUCATION,WALDORF,MD,PRIMARY,20601,19981011_8953,11385,611599535,20090,...,1,3,1773080,0,153876.0,23,7.0,4700,7.90,0


In [26]:
from mlflow import MlflowClient
client = MlflowClient()

In [28]:
name = "loan_approval"

In [30]:
client.get_latest_versions(name)

Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/get-latest-versions failed with error code 405 != 200. Response body: '<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>
'

In [31]:
client.get_model_version(name, 4)

Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


<ModelVersion: creation_timestamp=1681635678587, current_stage='Production', description='', last_updated_timestamp=1681635731001, name='loan_approval', run_id='c5a8ff3bb67542dc951b4ebf9fdeac75', run_link='', source='s3://fsd/4/c5a8ff3bb67542dc951b4ebf9fdeac75/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [32]:
client.get_model_version_download_uri(name, 4)

Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


's3://fsd/4/c5a8ff3bb67542dc951b4ebf9fdeac75/artifacts/model'

In [41]:
loaded_model.metadata.run_id

'c5a8ff3bb67542dc951b4ebf9fdeac75'

In [35]:
model_info.model_uri

'runs:/c5a8ff3bb67542dc951b4ebf9fdeac75/model'

In [42]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.pyfunc.model
  run_id: c5a8ff3bb67542dc951b4ebf9fdeac75

In [43]:
client.get_registered_model(name)

Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


<RegisteredModel: creation_timestamp=1681343018279, description='', last_updated_timestamp=1681635731001, latest_versions=[<ModelVersion: creation_timestamp=1681513922725, current_stage='None', description='', last_updated_timestamp=1681513922725, name='loan_approval', run_id='006466f154774bf0934876ffad21ecff', run_link='', source='s3://fsd/4/006466f154774bf0934876ffad21ecff/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>,
 <ModelVersion: creation_timestamp=1681635678587, current_stage='Production', description='', last_updated_timestamp=1681635731001, name='loan_approval', run_id='c5a8ff3bb67542dc951b4ebf9fdeac75', run_link='', source='s3://fsd/4/c5a8ff3bb67542dc951b4ebf9fdeac75/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>], name='loan_approval', tags={}>

In [47]:
rm = client.get_registered_model(name)

Unverified HTTPS request is being made to host 'keycloak.mlplatform.enfint.tech'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


ValueError: too many values to unpack (expected 2)

In [53]:
rm.latest_versions[-1].run_id

'c5a8ff3bb67542dc951b4ebf9fdeac75'